# LightGBM

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/langchain-tools/blob/main/examples/langchain-tools_LightGBM.ipynb)

In [ ]:
!pip install git+https://github.com/fuyu-quant/langchain-tools.git

In [ ]:
%%capture
!pip install langchain
!pip install openai
!pip install lightgbm

In [ ]:
import pandas as pd
import lightgbm as lgbm

# LangChain
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, tool


# langchaintools 
from langchaintools import toolmaker
from langchaintools import mltools


import os

os.environ["LANGCHAIN_HANDLER"] = "langchain"
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

### データセットの用意
* 現在の実装では学習や推論を行うのにcsvファイルにする必要があります

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
x = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target
print(f'全データ数:{len(x)}')

train, test, train_target, test_target = train_test_split(x, y ,test_size = 0.2, random_state=3655)

train['target'] = train_target
#test['target'] = test_target

print(f'学習データ数:{len(train)}')
print(f'学習データ数:{len(test)}')
train.to_csv('/content/train.csv')
test.to_csv('/content/test.csv')

### LightGBMの学習

In [ ]:
tools = [
    Tool(name = 'lgbm_train_tool',func = mltools.lgbm_train_tool, description=""),
    Tool(name = 'lgbm_inference_tool',func = mltools.lgbm_inference_tool, description="")
    ]

In [ ]:
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

agent.run("train.csvを使ってLightGBMの学習を行なってください")

In [ ]:
agent.run("test.csvをLightGBMで推論を行なってください")

In [ ]:
test_pred = pd.read_csv('/content/inference.csv', index_col = 0)

In [ ]:
test_target